In [1]:
import os
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import pycocotools
import tensorflow as tf
from pathlib import Path
print(tf.__version__)

2.6.0


In [2]:
DATA_DIR = Path("./data/20k")
TRAIN_FILE = DATA_DIR / "20k_train.tfrecord"
TEST_FILE = DATA_DIR / "20k_test.tfrecord"
CLASS_LABELS_FILE = DATA_DIR / "classlabels.txt"
CKPT_DIR = Path("./checkpoints") / "ckpts03"
MODEL_EXPORT_DIR = Path("./export") / "03"
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(MODEL_EXPORT_DIR, exist_ok=True)

In [3]:
with open(CLASS_LABELS_FILE, "r") as f:
    label_map = [s.strip() for s in f.readlines()]
    
label_map_dict = {}
for i, l in enumerate(label_map):
    label_map_dict[i+1] = l

for k, v in label_map_dict.items():
    print(k, ": ", v)

1 :  circle
2 :  diamond
3 :  halfcircle
4 :  heart
5 :  hexagon
6 :  hexstar
7 :  moon
8 :  octagon
9 :  pentagon
10 :  pentstar
11 :  plus
12 :  square
13 :  trapezoid
14 :  triangle


In [4]:
train_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TRAIN_FILE.resolve()),
    size = 18000,
    label_map = label_map_dict
)

test_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TEST_FILE.resolve()),
    size = 2000,
    label_map = label_map_dict
)

In [22]:




spec = object_detector.EfficientDetLite2Spec(
    model_dir = CKPT_DIR,
    hparams = hparams
)

2021-08-30 16:00:20.818910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1885] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2021-08-30 16:00:20.819422: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-30 16:00:21.433247: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7409 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


In [23]:
model = object_detector.create(
    train_data = train_data,
    model_spec = spec,
    validation_data = test_data,
    epochs = 500,
    batch_size = 64,
    train_whole_model = False,
    do_train = False,
)

In [7]:
tfmodel = model.create_model()

In [8]:
ckpt_dir = str(CKPT_DIR / "ckpt-500")
tfmodel.load_weights(ckpt_dir)

In [10]:
spec.export_saved_model(
    model = tfmodel,
    saved_model_dir = MODEL_EXPORT_DIR,
    batch_size = 64,
    pre_mode = None,
    post_mode = None,
)

ValueError: in user code:

    /home/ms/.pyenv/versions/tflite_model_maker/lib/python3.7/site-packages/tensorflow/python/saved_model/signature_serialization.py:159 signature_wrapper  *
        structured_outputs, signature_function.name, signature_key)
    /home/ms/.pyenv/versions/tflite_model_maker/lib/python3.7/site-packages/tensorflow/python/saved_model/signature_serialization.py:223 _normalize_outputs  **
        .format(value, key, compat.as_str_any(function_name), signature_key))

    ValueError: Got a non-Tensor value `[<tf.Tensor 'StatefulPartitionedCall:0' shape=(64, 56, 56, 126) dtype=float32>, <tf.Tensor 'StatefulPartitionedCall:1' shape=(64, 28, 28, 126) dtype=float32>, <tf.Tensor 'StatefulPartitionedCall:2' shape=(64, 14, 14, 126) dtype=float32>, <tf.Tensor 'StatefulPartitionedCall:3' shape=(64, 7, 7, 126) dtype=float32>, <tf.Tensor 'StatefulPartitionedCall:4' shape=(64, 4, 4, 126) dtype=float32>]` for key 'output_0' in the output of the function __inference___call___13766